# Data Ingestion

In [3]:
import os

In [1]:
%pwd

'/Users/zewen/Documents/Zewen-Yang/TextSummarizer/notebook'

## Change to the Main Path

In [4]:
os.chdir("../")
%pwd

'/Users/zewen/Documents/Zewen-Yang/TextSummarizer'

## Define a dataclass for `DataIngestionConfigurations`

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

## Define `ConfigurationManager` for `get_data_ingestion_config`

In [10]:
from TextSummarizer.constants import *
from TextSummarizer.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

## Define `DataIngestion` Function for Getting the Data via `DataIngestionConfig`

In [11]:
import os
import urllib.request as request
import zipfile
from TextSummarizer.logging import logger
from TextSummarizer.utils.common import get_size

In [12]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

## Get `artifacts` folder

In [17]:
import sys
from TextSummarizer.utils.exception import CustomException

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise CustomException(e, sys)

[2023-06-05 00:46:33,451] 30 TextSummarizerLogger - INFO - common: yaml file: config/config.yaml loaded successfully
[2023-06-05 00:46:33,453] 30 TextSummarizerLogger - INFO - common: yaml file: params.yaml loaded successfully
[2023-06-05 00:46:33,453] 50 TextSummarizerLogger - INFO - common: created directory at: artifacts
[2023-06-05 00:46:33,454] 50 TextSummarizerLogger - INFO - common: created directory at: artifacts/data_ingestion
[2023-06-05 00:46:34,882] 12 TextSummarizerLogger - INFO - 1679650260: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: ECA2:8ABE:197035B:1A